In [28]:
import glob
import os
import random
import logging
import pandas
import numpy
import pickle
import gensim
import operator
import re
from random import shuffle


from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from typing import Iterable

from gensim.models.word2vec import Word2Vec
from nltk import tokenize, casual_tokenize
import nltk 
from zipfile import ZipFile
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
from nltk.corpus import stopwords

In [6]:
model = Word2Vec.load('/home/jcai/geometry_of_law/word2vec_v20480_d256_shuffled_opinion/cc.w2v')

In [7]:
word_count_dict = {}

In [9]:
for word, vocab_obj in model.wv.vocab.items():
    word_count_dict[word] = vocab_obj.count

In [11]:
sif_dict = {}

In [12]:
for word, count in word_count_dict.items():
    sif_dict[word] = .001/(.001+count)

In [13]:
pk.dump(sif_dict, open( "/home/jcai/geometry_of_law/word2vec_v20480_d256_shuffled_opinion/cc_sif_dict.p", "wb" ))

In [18]:
# defines vocabulary size and vector dimension
voc_size = 20480
vec_dim  = 256
model_name = 'cc'
directory_name = 'word2vec_v20480_d256_shuffled_opinion'


# configues loggin
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO, filename=directory_name+'/'+model_name+'.log')
console = logging.StreamHandler()
console.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s : %(levelname)s : %(message)s')
console.setFormatter(formatter)
logging.getLogger('').addHandler(console)

# sets working directory
wd = '/home/jcai/geometry_of_law'
os.chdir(wd)
print(wd + ' is wd')

#read zipfiles
zfile_cc = ZipFile('/data/Data/Circuit_Courts/circuit-cases/sentences.zip', allowZip64 = True)
items_cc = [("cc", x) for x in zfile_cc.namelist()]
items_cc = [x for x in items_cc if '.txt' in x[1]]
items_cc = [x for x in items_cc if int(x[1].split("/")[1].split("_")[1]) in range(1970,2006)]

shuffle(items_cc)

/home/jcai/geometry_of_law is wd


In [22]:
#removing stopword
stopwords_year = [str(x) for x in list(range(1970,2005))]

#get judgenames
judge_bio = pd.read_stata("/data/Data/Judge-Bios/judgebios/JudgesBioReshaped_TOUSE.dta")

stopwords_judge_name = list(judge_bio['judgefirstname']) + list(judge_bio['judgelastname'])

#defines a tokenizer that does not save punctuation and convert everything to lower case
#also removes stopwords
stop = stopwords.words('english') + stopwords_year + stopwords_judge_name
stop = [x.lower() for x in stop]
stop = set(stop)

In [24]:
def tokenize_no_punct_all_lower(txt):
    txt_tokenize = casual_tokenize(txt,preserve_case=False,strip_handles=True)
    txt_tokenize = [word for word in txt_tokenize if re.sub(r"\-", "", word).isalpha()]
    txt_tokenize = [word for word in txt_tokenize if word not in stop]
    return txt_tokenize

In [55]:
vocab_set = set(model.wv.vocab.keys())

In [66]:
#defines iterator that splits by lines
def document_lines_vec(docnames):
    fname = docnames.split('/')[-1]
    txt = zfile_cc.open(docnames,"r").read()
    txt = txt.decode("utf-8")
    tokens = tokenize_no_punct_all_lower(txt)
    tokens_tagged = nltk.pos_tag(tokens,tagset='universal')
    tokens = [i[0] for i in tokens_tagged if i[1] in ["VERB","NOUN","ADJ","ADV"]]
    tokens = [t for t in tokens if t in vocab_set]
    sif_vec = np.mean([sif_dict[t]*model.wv[t] for t in tokens],axis=0)
    return sif_vec

In [64]:
sif_vec_dict = {}

In [ ]:
for item in items_cc:
    sif_vec_dict[item[1].split('/')[-1]] = document_lines_vec(item[1])

In [ ]:
pk.dump(sif_vec_dict, 
        open( "/home/jcai/geometry_of_law/word2vec_v20480_d256_shuffled_opinion/cc_sif_vec_dict.p", "wb" ))
print("saved")